In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from google.colab import drive

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/VisionSpec_QC/model_week2.keras")

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = "/content/drive/MyDrive/VisionSpec_QC/dataset/val/defect/WIN_20220329_17_20_20_Pro.jpg"
IMG_SIZE = (224, 224)

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

In [ ]:
last_conv_layer_name = "Conv_1"

base_model = model.get_layer("mobilenetv2_1.00_224")
classifier_layers = model.layers[1:]

def classifier_forward(x):
    for layer in classifier_layers:
        x = layer(x)
    return x

grad_model = tf.keras.models.Model(
    inputs=base_model.input,
    outputs=[
        base_model.get_layer(last_conv_layer_name).output,
        base_model.output
    ]
)

In [ ]:
with tf.GradientTape() as tape:
    conv_outputs, base_features = grad_model([img_array])
    tape.watch(conv_outputs)

    predictions = classifier_forward(base_features)
    class_index = tf.argmax(predictions[0])
    loss = predictions[:, class_index]

grads = tape.gradient(loss, conv_outputs)

In [ ]:
# ---- COMPUTE GRAD-CAM ----
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
conv_outputs = conv_outputs[0]

heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
heatmap = tf.squeeze(heatmap)
heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)

# ---- OVERLAY HEATMAP ----
img_cv = cv2.imread(img_path)
img_cv = cv2.resize(img_cv, IMG_SIZE)

heatmap_resized = cv2.resize(heatmap.numpy(), IMG_SIZE)
heatmap_uint8 = np.uint8(255 * heatmap_resized)
heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)

superimposed_img = cv2.addWeighted(img_cv, 0.6, heatmap_color, 0.4, 0)


plt.figure(figsize=(6,6))
plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Grad-CAM Explanation")
plt.savefig("gradcam_defect_example.png", bbox_inches="tight")
plt.show()

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Load your model
model = load_model("/content/drive/MyDrive/VisionSpec_QC/model_week2.keras")

# Path of the image you want to test
img_path = "/content/drive/MyDrive/VisionSpec_QC/dataset/val/defect/WIN_20220329_17_20_20_Pro.jpg"

# Read and preprocess the image
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # OpenCV loads in BGR
img = cv2.resize(img, (224, 224))             # change size to what your model was trained on
img = img / 255.0                             # normalize
img = np.expand_dims(img, axis=0)             # shape: (1, 224, 224, 3)

# Predict
prediction = model.predict(img)

print("Raw model output:", prediction)
if prediction[0][0] <= 0.5:
    print("Defect detected ❌")
else:
    print("Pass ✅")